In [ ]:
import json
import core_constants as cc
import functions as fx
import pandas as pd
import sqlite3 as sql
import recordlinkage
import queries

## Building out the Dataframes

In [ ]:
limit = True
ranked = True
query = queries.get_query_all247(limit, ranked)
df_247 = (fx.connDBAndReturnDF(query)).set_index('IDYR')
df_247.index.name = '247_IDYR'
df_247

In [ ]:
fuzzy = False
query = queries.get_query_UnlinkedNCAA(fuzzy)
df_ncaa = (fx.connDBAndReturnDF(query)).set_index('ID')
df_ncaa.index.name = 'NCAA_ID'
df_ncaa

In [ ]:
indexer = recordlinkage.Index()
indexer = recordlinkage.BlockIndex(on=['KeyPositionGroup'])


In [ ]:
pairs = indexer.index(df_ncaa, df_247)

In [ ]:
from functions import YearOther

sumFields = []
c = recordlinkage.Compare()
c.string('PlayerName', 'PlayerName', method='damerau_levenshtein', label='PlayerName')
sumFields.append('PlayerName')
c.exact('StandardizedPosition', 'StandardizedPosition', label='StandardizedPosition')
sumFields.append('StandardizedPosition')
c.exact('KeyPositionGroup', 'KeyPositionGroup', label='KeyPositionGroup')
sumFields.append('KeyPositionGroup')
c.add(YearOther('Year', 'Year', label='Year'))
sumFields.append('Year')


In [ ]:
features = c.compute(pairs, df_ncaa, df_247)

In [ ]:
sum = 0
for field in sumFields:
    sum = sum + features[field]

features['sum'] = sum / len(sumFields)

features = features[features['Year'] == 1.0]

features

In [ ]:
features.insert(0, 'sourceID', features.index.get_level_values(0))
features.insert(1, 'targetID', features.index.get_level_values(1))

filteredList = []
for idx, data in features.groupby(level=0):
    data = data.loc[data['sum'].idxmax()]
    if (data['sum'] > .96):
        filteredList.append(data)
dfFinal = pd.DataFrame()
dfFinal = dfFinal.append(filteredList)
dfFinal.to_csv("resultsNCAATransfer.csv")

In [ ]:
links = dfFinal.values.tolist()
recordCount = 0

for record in links:
    values = [record[0], record[1], 5, 3, record[5], 1]
    query = '''INSERT INTO RecordLinks(MasterID, TargetID, KeyDataSet, KeyLinkType, LinkConfidence, Transfer)
        VALUES (?,?,?,?,?,?)'''
    
    try:
        conn = sql.connect(cc.databaseName)
        c = conn.cursor()

        c.execute(query, values)
        conn.commit()
        recordCount = recordCount + 1
    except Exception as e:
        print('Database Commit Failure')
        print(e)

print('Database write of ' + str(recordCount) + ' records successful')



## Below is to create and save annotations

In [ ]:
fx.create_AnnotationFile(dfFinal, df_ncaa, df_247)

In [ ]:
filename = "..//Annotations//Results/result.json"
keydataset = 5
targetkeydataset = 1
transfer = 1

fx.save_Annotations(filename, keydataset, targetkeydataset, transfer)